# BC 4
## Flat ensemble of tiny networks
## Input = whole image

This conditions are ran on local machine as it exceeds the maximum RAM provided by Google Colab.

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
from keras.datasets import cifar10
from keras.utils import to_categorical
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import random
from scipy import stats
from sklearn.metrics import accuracy_score

In [ ]:
NUM_TN = 30

#dataset specific parameters
NUM_CLASS = 10
SCALE = 32

## Data Pre-Processing

In [ ]:
#load dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

In [ ]:
y_train = y_train.reshape(y_train.shape[0])
y_test = y_test.reshape(y_test.shape[0])

In [ ]:
#Generate subset of train image
X_train_subset_list = []
y_train_subset_list = []
train_zipped = zip(X_train, y_train)
train_zipped = np.array(tuple(train_zipped))

for i in range(NUM_TN):
  train_zipped_subset = random.choices(train_zipped, k=int(len(train_zipped)*60/100))
  X_train_subset = []
  y_train_subset = []
  for a in train_zipped_subset:
    X_train_subset.append(a[0])
    y_train_subset.append(a[1])
  X_train_subset = np.array(X_train_subset)
  y_train_subset = np.array(y_train_subset)

  X_train_subset_list.append(X_train_subset)
  y_train_subset_list.append(y_train_subset)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  """


In [ ]:
#normalizing dataset
for i in range(NUM_TN):
  X_train_subset_list[i] = X_train_subset_list[i]/255

X_test = X_test/255

In [ ]:
#transform to vector
for i in range(NUM_TN):
  X_train_subset_list[i] = X_train_subset_list[i].reshape((-1, SCALE*SCALE*3))

X_test = X_test.reshape((-1, SCALE*SCALE*3))

In [ ]:
#one-hot encoding
for i in range(NUM_TN):
  y_train_subset_list[i] = to_categorical(y_train_subset_list[i])

y_test_one_hot = to_categorical(y_test)

## Model

In [ ]:
#build ANN model
ensemble = []
for i in range(NUM_TN):
  model = Sequential()
  model.add(Dense(64, activation='relu', input_dim=SCALE*SCALE*3))
  model.add(Dense(64, activation='relu'))
  model.add(Dense(NUM_CLASS, activation = 'softmax'))
  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
  ensemble.append(model)

In [ ]:
#train model
history = []
for i in range(NUM_TN):
  print('Model %d' %i)
  hist = ensemble[i].fit(X_train_subset_list[i], y_train_subset_list[i], validation_data=(X_test, y_test_one_hot), epochs=5, batch_size=128)
  history.append(hist)

## Model Evaluation

In [ ]:
# Predict on test image
y_pred_list = []
for i in range(NUM_TN):
  y_pred = ensemble[i].predict(X_test)
  y_pred_list.append(y_pred)

# Ensemble voting
y_pred_list = np.array(y_pred_list)
y_pred_list = np.argmax(np.sum(y_pred_list, axis=0), axis=1)

In [ ]:
accuracy = accuracy_score(y_test, y_pred_list)
accuracy